In [305]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import matplotlib.pyplot as plt
import torchtext

import tiktoken
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
import pickle
import ast

In [306]:
#GLOBALS

block_size = 256 #This is the value of T

In [314]:
import os

#dataset https://nlp.stanford.edu/projects/nmt/

#tiktoken api https://github.com/openai/tiktoken
cl100k_base = tiktoken.get_encoding("cl100k_base")

# In production, load the arguments directly instead of accessing private attributes
# See openai_public.py for examples of arguments for specific encodings
tokenizer = tiktoken.Encoding(
    # If you're changing the set of special tokens, make sure to use a different name
    # It should be clear from the name what behaviour to expect.
    name="cl100k_im",
    pat_str=cl100k_base._pat_str,
    mergeable_ranks=cl100k_base._mergeable_ranks,
    special_tokens={
        **cl100k_base._special_tokens,
        "<|PAD|>": 100277,
        "<|START|>": 100278,
        "<|END|>": 100279,
    }
)
print(tokenizer.n_vocab) #this is the number of tokens in our tokenizer
print(tokenizer._special_tokens) #prints out our special tokens 

specials = {"<|PAD|>","<|START|>","<|END|>"}

100280
{'<|endoftext|>': 100257, '<|fim_prefix|>': 100258, '<|fim_middle|>': 100259, '<|fim_suffix|>': 100260, '<|endofprompt|>': 100276, '<|PAD|>': 100277, '<|START|>': 100278, '<|END|>': 100279}


In [296]:
def create_dataset():
    num_examples = 1000

    en_max = 0 
    with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
        idx_en = []
        sentences_en = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            sentence = tokenizer(line)
            if len(sentence) > en_max:
                en_max = len(sentence)
                print(en_max)
            sentence = ["<|PAD|>"] + sentence + ["<|PAD|>"]

            if len(sentence) <= block_size:
                len_pad = block_size - len(sentence)
                sentence = sentence + len_pad*"<|PAD|> ".split()
                assert len(sentence) == block_size, print(len(sentence))
                #idx_en.append(i)
                sentences_en.append(sentence)
            else:
                sentences_en.append("|<DEL>|")

    print(en_max)    
    print(f"Length of sentences: {len(sentences_en)}")

    de_max = 0 
    with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
        idx_de = []
        sentences_de = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            sentence = tokenizer(line)
            if len(sentence) > de_max:
                de_max = len(sentence)
                print(de_max)

            sentence = ["<|START|>"] + sentence + ["<|END|>"]
            if len(sentence) <= block_size:
                len_pad = block_size - len(sentence)
                sentence = sentence + len_pad*"<|PAD|> ".split()
                assert len(sentence) == block_size, print(len(sentence))
                #idx_de.append(i)
                sentences_de.append(sentence)
            else:
                sentences_de.append("|<DEL>|")
                
    print(de_max)  
    print(f"Length of sentences: {len(sentences_de)}")

    sentences = np.array([sentences_en, sentences_de])

    idx = np.where(sentences == "|<DEL>|")
    sentences = np.delete(sentences, idx[1], axis = 1)

    sentences_en = sentences[0]
    sentences_de = sentences[1]

    print(f"Length of new english sentences: {len(sentences_en)}")
    print(f"Length of new german sentences: {len(sentences_de)}")

    with open(os.getcwd()+'\\data\\english_sentences.pkl', 'wb') as f:
        pickle.dump(sentences_en, f)

    with open(os.getcwd()+'\\data\\german_sentences.pkl', 'wb') as f:
        pickle.dump(sentences_de, f)

In [297]:
create = True
if create:
    create_dataset()

  0%|          | 0/1000 [00:00<?, ?it/s]

32
34
47
49
54
57
58
67
87
91
92
97
98
100
100
Length of sentences: 1000


  0%|          | 0/1000 [00:00<?, ?it/s]

34
43
48
63
64
71
75
86
98
100
100
Length of sentences: 1000
Length of new english sentences: 1000
Length of new german sentences: 1000


In [298]:
#TRAIN AND VAL DATASETS

with open(os.getcwd()+'\\data\\english_sentences.pkl', 'rb') as f:
    english_sentences = pickle.load(f)

with open(os.getcwd()+'\\data\\german_sentences.pkl', 'rb') as f:
    german_sentences = pickle.load(f)

In [299]:
"|<DEL>|" in english_sentences

False

In [300]:
english_sentences.shape

(1000, 256)

In [50]:
#BATCH LOADER
n = int(0.9*len(df))


rough work

In [301]:
encoder = tiktoken.get_encoding("cl100k_base")


In [304]:
encoding.encode(["tiktoken is great!"])

TypeError: expected string or buffer

In [303]:
encoding.decode([83, 1609, 5963, 374, 2294, 0])

'tiktoken is great!'

In [315]:
num_examples = 5

en_max = 0 
with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
    idx_en = []
    sentences_en = []
    for i in tqdm(range(num_examples)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        sentence = "<|PAD|> " + (line) + " <|PAD|>"
        print(sentence)
        print(tokenizer.encode(sentence, allowed_special = specials))
        # if len(sentence) > en_max:
        #     en_max = len(sentence)
        #     print(en_max)
        # sentence = ["<|PAD|>"] + sentence + ["<|PAD|>"]

  0%|          | 0/5 [00:00<?, ?it/s]

<|PAD|> iron cement is a ready for use paste which is laid as a fillet by putty knife or finger in the mould edges ( corners ) of the steel ingot mould . <|PAD|>
[100277, 11245, 24532, 374, 264, 5644, 369, 1005, 25982, 902, 374, 17551, 439, 264, 1488, 1169, 555, 2231, 1919, 22145, 477, 14654, 304, 279, 51370, 13116, 320, 24359, 883, 315, 279, 9699, 6892, 354, 51370, 662, 220, 100277]
<|PAD|> iron cement protects the ingot against the hot , abrasive steel casting process . <|PAD|>
[100277, 11245, 24532, 36236, 279, 6892, 354, 2403, 279, 4106, 1174, 94804, 9699, 25146, 1920, 662, 220, 100277]
<|PAD|> a fire restant repair cement for fire places , ovens , open fireplaces etc . <|PAD|>
[100277, 264, 4027, 2800, 519, 13023, 24532, 369, 4027, 7634, 1174, 297, 21778, 1174, 1825, 4027, 27170, 5099, 662, 220, 100277]
<|PAD|> Construction and repair of highways and ... <|PAD|>
[100277, 24987, 323, 13023, 315, 60395, 323, 2564, 220, 100277]
<|PAD|> An announcement must be commercial character . <